In [ ]:
# Importações necessárias
import pyarrow.parquet as pq
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as sch

##Dask
import json
from dask import bag as db
from dask import dataframe as dd

# Importações de modelos e métricas do Scikit-Learn
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import (mean_squared_error, confusion_matrix, precision_score, 
                             recall_score, f1_score, roc_curve, auc, accuracy_score, 
                             silhouette_score, adjusted_rand_score)
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline

# Algoritmos de machine learning
from sklearn import ensemble, linear_model, neighbors, tree, naive_bayes, svm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.cluster import KMeans, AgglomerativeClustering

# Configuração do pandas para exibição de colunas
pd.set_option('display.max_columns', 500)

# Supressão de warnings
import warnings
warnings.filterwarnings("ignore")

# Leitura dos dados
members = pq.read_table('/kaggle/input/case-data-master-2024/members.parquet')
transactions = pq.read_table('/kaggle/input/case-data-master-2024/transactions.parquet')
logs = pq.read_table('/kaggle/input/case-data-master-2024/user_logs.parquet')

# Conversão para DataFrames pandas
members_pd = members.to_pandas()
transactions_pd = transactions.to_pandas()
logs_pd = logs.to_pandas()

# Liberação de memória
del members
del transactions
del logs

In [ ]:
# Visualização inicial das primeiras linhas e tipos de dados
print(members_pd.head())
print(members_pd.dtypes)

In [ ]:
# Exibição de contagem de valores únicos por coluna
for i in members_pd.columns:
    print(i)
    print(len(members_pd[i].unique()))
    print("")

In [ ]:
# Convertendo a coluna 'bd' para numérico, forçando erros para NaN
members_pd['bd'] = pd.to_numeric(members_pd['bd'], errors='coerce')

# Removendo idades inválidas e substituindo-as por NaN
members_pd['bd'] = members_pd['bd'].apply(lambda x: x if 0 < x < 100 else np.nan)

# Transformação da coluna de registro de data para tipo datetime
members_pd['registration_init_time'] = pd.to_datetime(members_pd['registration_init_time'], format='%Y%m%d')

# Calculando o tempo desde o cadastro em dias
last_date = pd.to_datetime('2017-03-31')  # Última data conhecida nos dados
members_pd['days_since_registration'] = (last_date - members_pd['registration_init_time']).dt.days

# Visualizando as mudanças e o tratamento inicial de outliers
print(members_pd[['city', 'registered_via', 'bd', 'days_since_registration']].head())

In [ ]:
# Visualizando as primeiras linhas e checando tipos de dados
transactions_pd.head()

In [ ]:
#Checando tipos de dados
transactions_pd.dtypes

In [ ]:
for i in transactions_pd.columns:
    print(i)
    print(len(transactions_pd[i].unique()))
    print("")

In [ ]:
# Convertendo as colunas para o tipo datetime
transactions_pd.transaction_date = pd.to_datetime(transactions_pd.transaction_date, format='%Y%m%d', errors='coerce')
transactions_pd.membership_expire_date = pd.to_datetime(transactions_pd.membership_expire_date, format='%Y%m%d', errors='coerce')

# Calculando os dias até a expiração da assinatura
transactions_pd['transaction_to_expire_days'] = (transactions_pd.membership_expire_date - transactions_pd.transaction_date).dt.days

# Verificando as primeiras linhas após a adição da nova coluna
print(transactions_pd[['transaction_date', 'membership_expire_date', 'transaction_to_expire_days']].head())


In [ ]:
# Corrigindo valores negativos em 'total_secs' e mantendo valores positivos
logs_pd['total_secs'] = np.where(logs_pd['total_secs'] < 0, 0, logs_pd['total_secs'])

# Visualizando as primeiras linhas após o ajuste
print(logs_pd.head())

In [ ]:
# Gerando estatísticas descritivas para validar as alterações
print(logs_pd.describe())

### Lógica Temporal da Target e Features

A estratégia dos executivos é prever a saída da carteira de clientes, o que implica em algumas considerações importantes:

Uso de Dados Atuais: Durante a implementação do modelo, utilizaremos dados recentes de cadastro e de consumo, abrangendo desde os registros mais atuais até dados históricos, para prever a rentabilidade futura dos clientes.
Avaliação da Performance em M+1: O objetivo mencionado no texto é avaliar a performance no mês seguinte (M+1). Isso sugere que buscamos prever a rentabilidade dos clientes no mês subsequente.
Validação Out-of-Time (OOT): Para a validação da performance do modelo, propomos realizar um deslocamento (shift) de 1 mês à frente para o conjunto de teste. Essa abordagem permitirá avaliar a eficácia do modelo na previsão da rentabilidade futura de maneira robusta.